In [201]:
import tensorflow as tf
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin

In [202]:
f=open("d:/deep/sis/annotation.txt")
text=f.readlines()
words=[]
dic=set()
for line in text:
    word=line.split()
    for w in word:
        if w.isalnum():
            dic.add(w)
            words.append(w)
f.close()

In [203]:
f=open("d:/deep/caption/annotation.txt")
text=f.readlines()
for line in text:
    word=line.split()
    for w in word:
        if w.isalnum():
            dic.add(w)
            words.append(w)
f.close()

In [204]:
f=open("d:/deep/annotations/captions.txt")
text=f.readlines()
for line in text:
    word=line.split()
    for w in word:
        w="".join(l for l in w if l not in {',','.','!','?','\"','(',')','\''})
        if len(w)>0:
            dic.add(w)
            words.append(w)
f.close()

In [205]:
embedding_size=50
vocabulary_size=len(dic)

In [206]:
vocabulary_size

37119

In [207]:
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

In [208]:
import collections
import math
def build_dataset(words, vocabulary_size):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

In [209]:
# Step 2: Build the dictionary and replace rare words with UNK token.
data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)
#del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 1], ('a', 275859), ('the', 210048), ('to', 87244), ('and', 80056)]
Sample data [50, 3372, 235, 7, 212, 9, 24, 3, 61, 4051] ['our', 'landmark', 'tree', 'in', 'town', 'was', 'about', 'to', 'be', 'destroyed']


In [210]:
count[:30]

[['UNK', 1],
 ('a', 275859),
 ('the', 210048),
 ('to', 87244),
 ('and', 80056),
 ('of', 75072),
 ('is', 73801),
 ('in', 66970),
 ('on', 49809),
 ('was', 44121),
 ('with', 37324),
 ('man', 35008),
 ('we', 33085),
 ('it', 27801),
 ('for', 27763),
 ('i', 27215),
 ('are', 25927),
 ('they', 20513),
 ('were', 20235),
 ('at', 20062),
 ('this', 19063),
 ('there', 18282),
 ('woman', 18210),
 ('some', 18144),
 ('about', 16757),
 ('an', 16744),
 ('people', 16536),
 ('had', 15488),
 ('that', 14623),
 ('my', 14203)]

In [211]:
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

In [212]:
import collections
import math
import os
import random
import zipfile

import numpy as np
# Step 3: Function to generate a training batch for the skip-gram model.


batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3372 landmark -> 235 tree
3372 landmark -> 50 our
235 tree -> 3372 landmark
235 tree -> 7 in
7 in -> 212 town
7 in -> 235 tree
212 town -> 9 was
212 town -> 7 in


In [213]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()


In [214]:

# Step 5: Begin training.
num_steps = 50001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  271.182281494
Nearest to are: theory, elmo, daog, revived, prerequisite, campout, intermingling, in,
Nearest to is: diaphragm, mised, contemplating, printout, healthy, josephs, miniskirt, committed,
Nearest to from: congo, jet, wildnerness, cooldown, burden, joes, dramtic, caravans,
Nearest to down: petitioned, smiliing, streaked, blob, flows, sweltering, cc, stip,
Nearest to food: encased, operates, mettle, creeped, reached, paramount, occasions, costumers,
Nearest to our: vava, drip, aft, hilleren, resort, goor, barber, dignified,
Nearest to his: bouncing, caressing, heels, beautified, concluding, surroundings, leaflet, solicit,
Nearest to playing: linguine, prarrie, thumpers, curves, zepplin, dsplay, butterflies, connects,
Nearest to girl: tup, beanbag, quilts, muttered, clearity, know, forager, fly,
Nearest to an: dart, heartstrings, kittens, percy, willow, porter, plated, crystallization,
Nearest to night: malfunctioned, riddled, char, sandwi

In [111]:


# Step 6: Visualize the embeddings.


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

In [21]:
type(final_embeddings)

numpy.ndarray

In [22]:
final_embeddings[1:10]
reverse_dictionary[3]

'to'

In [23]:
final_embeddings[10]

array([ -9.36699584e-02,   1.21167980e-01,  -8.30498263e-02,
         1.16637692e-01,  -4.64468561e-02,   2.46119171e-01,
         1.49763569e-01,  -1.02460526e-01,  -9.48668942e-02,
        -4.22165506e-02,  -4.31381678e-03,   2.01981217e-02,
         6.93978965e-02,  -9.57138389e-02,   6.60868436e-02,
        -1.43870905e-01,  -3.83234173e-02,  -3.63396853e-02,
        -2.43188068e-02,   6.52682930e-02,  -1.67282879e-01,
         6.45207837e-02,  -2.45701801e-02,   1.09216124e-01,
         1.57300964e-01,  -1.16725154e-01,   7.93971028e-03,
        -3.04253120e-02,  -1.64908111e-01,  -4.93514277e-02,
        -6.08402006e-02,   3.44628915e-02,  -2.33122203e-02,
         3.87568586e-02,   2.88353302e-02,   3.82286264e-03,
        -6.03013150e-02,  -4.58969828e-03,   1.43634319e-01,
         6.40476048e-02,  -3.99349406e-02,  -5.35916165e-03,
         2.09251940e-01,  -2.77392752e-02,   9.25252512e-02,
         9.88240167e-03,  -1.67645365e-01,   2.89181306e-04,
        -8.67744163e-02,

In [24]:
x=final_embeddings

In [25]:
len(final_embeddings[0])

128

In [26]:
y=final_embeddings.transpose()

In [27]:
type(x)

numpy.ndarray

In [28]:
type(y)

numpy.ndarray

In [29]:
xx=np.matrix(x)
yy=np.matrix(y)

In [31]:
del z

In [74]:
dat=z[5]
r=dat.argsort()

In [78]:
r[0][0]

matrix([[20298, 22120, 16766, ...,   198,    14,     5]], dtype=int64)

In [68]:
for i in range(50,60):
    dat=z[i]
    r=dat.argsort()
    for j in range(vocabulary_size-10,vocabulary_size-1):
        print(r[0][j])
        
    

IndexError: index 25657 is out of bounds for axis 0 with size 1

In [59]:
type(rank)

NoneType

In [79]:
z[0][0]

matrix([[ 0.99999994,  0.2599456 ,  0.27113849, ...,  0.23150526,
          0.10193338,  0.18941617]], dtype=float32)

In [119]:
ww=np.asarray(z)

In [120]:
ww

array([[  1.00000024e+00,   2.05893189e-01,   9.81881395e-02, ...,
          6.70303628e-02,  -3.75667401e-03,  -1.02838114e-01],
       [  2.05893189e-01,   1.00000060e+00,   6.52542233e-01, ...,
          2.75734574e-01,   2.65183181e-01,   1.21470630e-01],
       [  9.81881395e-02,   6.52542233e-01,   9.99999344e-01, ...,
          1.14936277e-01,   1.92138731e-01,   1.25528365e-01],
       ..., 
       [  6.70303553e-02,   2.75734603e-01,   1.14936247e-01, ...,
          9.99999821e-01,   6.21179643e-04,   2.76152939e-01],
       [ -3.75666865e-03,   2.65183121e-01,   1.92138746e-01, ...,
          6.21179643e-04,   1.00000024e+00,   1.03790171e-01],
       [ -1.02838121e-01,   1.21470638e-01,   1.25528365e-01, ...,
          2.76152909e-01,   1.03790164e-01,   9.99999881e-01]], dtype=float32)

In [89]:
dat=ww[50]
rw=dat.argsort()


AttributeError: module 'numpy' has no attribute 'reverse'

In [90]:
arr =rw[::-1]

In [91]:
arr

array([   50,  6894,  9838, ..., 22419, 13193, 21224], dtype=int64)

In [248]:
i=108
print(reverse_dictionary[i]+":")
dat=ww[i]
rw=dat.argsort()
arr =rw[::-1]
for x in arr[1:10]:
    print(reverse_dictionary[x])

has:


NameError: name 'ww' is not defined

In [96]:
reverse_dictionary[50]

'from'

In [128]:
dictionary['music']

108

In [233]:
f=open("d:/deep/annotations/captions_on_images.txt")
text=f.readlines()
num=0;
cap={}
for line in text:
    if (num==0):
        im_id=int(line)
        cap[im_id]=""
        num=1
        continue
    if (line[0]=='*'):
        num=0
        continue
    cap[im_id]+=line
f.close()

In [249]:
category={'accessory',
 'animal',
 'appliance',
 'electronic',
 'food',
 'furniture',
 'indoor',
 'rush',
 'outdoor',
 'human',
 'sports',
 'vehicle'}

In [235]:
type(category)

set

In [236]:
cap[42]

'this wire metal rack holds several pairs of shoes and sandals\na dog sleeping on a show rack in the shoes.\nvarious slides and other footwear rest in a metal basket outdoors.\na small dog is curled up on top of the shoes\na shoe rack with some shoes and a dog sleeping on them\n'

In [253]:
c_id=[]
for x in category:
    c_id.append(dictionary[x])

In [238]:
vote=[[0]*12]
vote=vote[0]
vote

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [239]:
vec=final_embeddings[0]
vec2=final_embeddings[1]
x=vec*vec2
sum(x)

0.19309834533760295

In [240]:
num=0
f_c={}
for idx in cap.keys():
    print(str(num)+" round")
    num+=1
    lis=cap[idx].split()
    vote=[[0]*12]
    vote=vote[0]
    for word in lis:
        w="".join(l for l in word if l not in {',','.','!','?','\"','(',')','\''})
        try:
            w_id=dictionary[w]
        except:
            continue
        vec=final_embeddings[w_id]
        max_sim=0
        for i in range(0,12):
            cate=c_id[i]
            vec2=final_embeddings[cate]
            sim_vec=vec*vec2
            cos_sim=sum(sim_vec)
            if (cos_sim>max_sim):
                max_cat=i
                max_sim=cos_sim
        vote[max_cat]+=1
    f_c[idx]=vote.index(max(vote))

0 round
1 round
2 round
3 round
4 round
5 round
6 round
7 round
8 round
9 round
10 round
11 round
12 round
13 round
14 round
15 round
16 round
17 round
18 round
19 round
20 round
21 round
22 round
23 round
24 round
25 round
26 round
27 round
28 round
29 round
30 round
31 round
32 round
33 round
34 round
35 round
36 round
37 round
38 round
39 round
40 round
41 round
42 round
43 round
44 round
45 round
46 round
47 round
48 round
49 round
50 round
51 round
52 round
53 round
54 round
55 round
56 round
57 round
58 round
59 round
60 round
61 round
62 round
63 round
64 round
65 round
66 round
67 round
68 round
69 round
70 round
71 round
72 round
73 round
74 round
75 round
76 round
77 round
78 round
79 round
80 round
81 round
82 round
83 round
84 round
85 round
86 round
87 round
88 round
89 round
90 round
91 round
92 round
93 round
94 round
95 round
96 round
97 round
98 round
99 round
100 round
101 round
102 round
103 round
104 round
105 round
106 round
107 round
108 round
109 round
110 round


In [254]:
num=0
f_c={}
for idx in cap.keys():
    print(str(num)+" round")
    num+=1
    lis=cap[idx].split()
    vote=[[0]*12]
    vote=vote[0]
    for word in lis:
        w="".join(l for l in word if l not in {',','.','!','?','\"','(',')','\''})
        try:
            w_id=dictionary[w]
        except:
            continue
        vec=final_embeddings[w_id]
        for i in range(0,12):
            cate=c_id[i]
            vec2=final_embeddings[cate]
            sim_vec=vec*vec2
            cos_sim=sum(sim_vec)
            vote[i]+=cos_sim
    f_c[idx]=vote.index(max(vote))

0 round
1 round
2 round
3 round
4 round
5 round
6 round
7 round
8 round
9 round
10 round
11 round
12 round
13 round
14 round
15 round
16 round
17 round
18 round
19 round
20 round
21 round
22 round
23 round
24 round
25 round
26 round
27 round
28 round
29 round
30 round
31 round
32 round
33 round
34 round
35 round
36 round
37 round
38 round
39 round
40 round
41 round
42 round
43 round
44 round
45 round
46 round
47 round
48 round
49 round
50 round
51 round
52 round
53 round
54 round
55 round
56 round
57 round
58 round
59 round
60 round
61 round
62 round
63 round
64 round
65 round
66 round
67 round
68 round
69 round
70 round
71 round
72 round
73 round
74 round
75 round
76 round
77 round
78 round
79 round
80 round
81 round
82 round
83 round
84 round
85 round
86 round
87 round
88 round
89 round
90 round
91 round
92 round
93 round
94 round
95 round
96 round
97 round
98 round
99 round
100 round
101 round
102 round
103 round
104 round
105 round
106 round
107 round
108 round
109 round
110 round


In [255]:
d_id=[]
c_name=[]
for x in category:
    d_id.append(dictionary[x])
    c_name.append(x)

In [256]:
print(c_id)
print(d_id)

[3075, 1938, 973, 591, 522, 10234, 7090, 326, 1292, 2269, 1808, 75]
[3075, 1938, 973, 591, 522, 10234, 7090, 326, 1292, 2269, 1808, 75]


In [257]:
f_c[42]

7

In [242]:
lis=cap[42].split()
vote=[[0]*12]
vote=vote[0]
vote_cat=vote
print(vote_cat)
for word in lis:
    w="".join(l for l in word if l not in {',','.','!','?','\"','(',')','\''})
    w_id=dictionary[w]
    vec=final_embeddings[w_id]
    max_sim=0
    for i in range(0,12):
        cate=c_id[i]
        vec2=final_embeddings[cate]
        sim_vec=vec*vec2
        cos_sim=sum(sim_vec)
        vote_cat[i]+=cos_sim
vote_cat

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[12.672057497887664,
 12.848335183975053,
 12.60769306026242,
 13.280746735530499,
 12.541408791590243,
 10.057129948354785,
 13.903195660693072,
 12.329704708454415,
 11.838915523132826,
 13.011326002876771,
 13.76926127933541,
 14.12976913709781]

In [258]:
dictionary['dog']

187

In [259]:
dictionary['animal']

522

In [260]:
dictionary['food']

75

In [261]:
vec=final_embeddings[186]
vec2=final_embeddings[75]
x=vec*vec2
sum(x)

0.28196703157836822

In [262]:
fin1=open('D:/deep/annotations/annotated.txt')
fin2=open('D:/deep/annotations/out_id.txt')
classes=fin1.readlines()
ids=fin2.readlines()
num=0;
for i in range(0,len(ids)):
    idx=int(ids[i])
    predict=c_name[f_c[idx]]
    label=classes[i][:-1]
    if (predict==label):
        num+=1
    else:
        print("pr: "+predict)
        print(label)
fin1.close()
fin2.close()

pr: sports
animal
pr: food
animal
pr: indoor
vehicle
pr: indoor
animal
pr: food
animal
pr: indoor
vehicle
pr: rush
animal
pr: animal
furniture
pr: rush
indoor
pr: electronic
animal
pr: rush
animal
pr: food
person
pr: outdoor
vehicle
pr: food
vehicle
pr: animal
indoor
pr: indoor
furniture
pr: furniture
animal
pr: animal
outdoor
pr: food
outdoor
pr: rush
furniture
pr: furniture
vehicle
pr: outdoor
food
pr: rush
animal
pr: animal
vehicle
pr: food
animal
pr: indoor
animal
pr: animal
indoor
pr: furniture
vehicle
pr: food
person
pr: food
furniture
pr: indoor
vehicle
pr: electronic
outdoor
pr: indoor
furniture
pr: animal
outdoor
pr: rush
vehicle
pr: indoor
animal
pr: indoor
outdoor
pr: animal
outdoor
pr: sports
vehicle
pr: furniture
indoor
pr: food
indoor
pr: rush
animal
pr: indoor
animal
pr: food
animal
pr: food
furniture
pr: indoor
furniture
pr: indoor
animal
pr: indoor
animal
pr: electronic
vehicle
pr: indoor
sports
pr: outdoor
food
pr: indoor
vehicle
pr: food
animal
pr: sports
furniture
p

In [263]:
type(text)

list

In [264]:
num

856

In [273]:
fin3=open('D:/deep/annotations/cluster.txt')
cluster=fin3.readlines()
num=0;
vot=[]
for i in range(0,12):
    vote2=[[0]*12]
    vote2=vote2[0]
    vot.append(vote2)
f_d={}
for i in range(0,len(ids)):
    idxx=int(ids[i])
    idx=int(cluster[i])
    vot[idx][f_c[idxx]]+=1
fin3.close()


In [274]:
for i in range(0,len(ids)):
    idxx=int(ids[i])
    idx=int(cluster[i])
    my=vot[idx]
    maxn=my.index(max(my))
    f_d[idxx]=c_name[maxn]

In [275]:
vot

[[6, 241, 3, 0, 39, 0, 0, 12, 0, 7, 0, 22],
 [49, 141, 5, 0, 39, 0, 0, 17, 0, 12, 4, 103],
 [37, 127, 15, 15, 172, 4, 0, 25, 1, 14, 7, 69],
 [42, 92, 10, 0, 33, 0, 0, 64, 1, 32, 13, 67],
 [6, 170, 81, 2, 5, 0, 0, 38, 0, 4, 7, 10],
 [48, 110, 3, 0, 78, 6, 0, 8, 0, 3, 1, 100],
 [44, 49, 6, 1, 70, 0, 0, 21, 0, 77, 4, 5],
 [19, 90, 5, 0, 11, 0, 0, 14, 1, 93, 7, 21],
 [74, 155, 38, 8, 145, 0, 0, 88, 3, 27, 9, 179],
 [66, 257, 38, 4, 58, 2, 0, 34, 0, 48, 13, 140],
 [39, 170, 12, 0, 48, 0, 0, 19, 4, 57, 12, 151],
 [8, 5, 12, 6, 17, 0, 0, 30, 0, 4, 0, 119]]

In [276]:
fin1=open('D:/deep/annotations/annotated.txt')
fin2=open('D:/deep/annotations/out_id.txt')
classes=fin1.readlines()
ids=fin2.readlines()
num=0;
for i in range(0,len(ids)):
    idx=int(ids[i])
    predict=f_d[idx]
    label=classes[i][:-1]
    if (predict==label):
        num+=1
    else:
        print("pr: "+predict)
        print(label)
fin1.close()
fin2.close()

pr: food
animal
pr: indoor
animal
pr: indoor
vehicle
pr: indoor
animal
pr: food
furniture
pr: indoor
animal
pr: indoor
vehicle
pr: indoor
animal
pr: indoor
furniture
pr: indoor
animal
pr: indoor
animal
pr: indoor
person
pr: animal
vehicle
pr: food
indoor
pr: indoor
vehicle
pr: food
indoor
pr: indoor
animal
pr: animal
outdoor
pr: indoor
outdoor
pr: furniture
vehicle
pr: indoor
animal
pr: food
indoor
pr: animal
vehicle
pr: indoor
animal
pr: indoor
animal
pr: food
indoor
pr: animal
vehicle
pr: indoor
person
pr: food
furniture
pr: indoor
vehicle
pr: indoor
outdoor
pr: animal
outdoor
pr: furniture
vehicle
pr: indoor
animal
pr: indoor
animal
pr: indoor
outdoor
pr: animal
outdoor
pr: furniture
vehicle
pr: food
indoor
pr: indoor
animal
pr: indoor
animal
pr: indoor
animal
pr: indoor
animal
pr: indoor
animal
pr: indoor
animal
pr: indoor
animal
pr: indoor
vehicle
pr: indoor
sports
pr: indoor
vehicle
pr: indoor
animal
pr: food
furniture
pr: indoor
vehicle
pr: indoor
animal
pr: indoor
animal
pr: in

In [277]:
num

878